Problem One

PLEASE NOTE: I HAVE SHIFTED THE STEP OF EACH $Y_N$ UP BY ONE, SO EX. $Y_N$ IN THE PROBLEM IS NOW $Y_{N+1}$ IN MY WORK

We can rearrange and rewrite to show that the truncation error $t^n = y_{n+2} - 4y_{n+1} + 3y_n + 2h(y_n')$. Rewrite in terms of functions of t...

$t^n = y(t+2h) - 4y(t+h) + 3y(t) + 2h(y'(t))$ Taylor expand...

$= y(t) + 2hy'(t) + 2h^2y''(t) + \frac{4}{3}h^3y'''(t) + O(h^4) -4y(t) -4hy'(t) - 2h^2y''(t) -\frac{2}{3}h^3y'''(t) - O(h^4) + 3y(t) + 2hy'(t)$ Simplify and cancel terms...

$t^n = \frac{2}{3}h^3y'''(t) + O(h^4)$, our final local truncation error.

We then want to prove the method is unstable. Consider a problem where the true solution is one. Set our intitial $y_{n+1} = 1 y_n = 2$. Let our stepsize be arbitrary. Our first step would give $y_{n+1} = -3 - 2hy_n'$, where $h$ is small. Our next step would have $y_n = 1, y_{n+1} = -3 - 2hy_n'$, so we would get $y_{n+2} = -12 - 10hy_n' -3$. We can continue this pattern if we wish, but clearly the method rapidly diverges away from 1. An example using arbitrarily small $h$ (such that the $hy_n'$ terms can be treated as 0) is below.

In [1]:
x0 = 2
x1 = 1
x2 = 0
y0 = 0
y1 = 0
y2 = 0
for i in range(0, 10):
    x2 = 4*x1 - 3*x0
    y0 = x0
    y1 = x1
    y2 = x2
    x0 = y1
    x1 = y2
    print(y2)

-2
-11
-38
-119
-362
-1091
-3278
-9839
-29522
-88571


Problem 2

First we perform the strong root test. So we can write $y_{n+2} - \frac{1}{2}(y_{n+1} + y_n) = 0$, which means the characteristic polynomial $p(\xi) = \xi^2 - 1/2\xi - 1/2$. So from this we get $\xi_1 = -1/2, \xi_2 = 1$. Following the strong root test, since the absolute value of both is less than or equal to 1, the method is zero-stable.

Checking consistency, we also need to look at the other characteristic function $\sigma(t) = \frac{1}{4}(4t^2 - t + 3)$. We want to see that $p'(1) = \sigma(1)$. That comes to $2 - 1/2 = \frac{1}{4}(4 - 1 + 3) \to \frac{3}{2} = \frac{6}{4}$, which is true. So the method is consistent.

Alternately, to prove consistency another way... 

Rearrange and change to functions to get $t^n = y(t+2h) - y(t+h)/2 - y(t)/2 - \frac{h}{4}(4y'(t+2h) - y'(t+h) + 3y'(t)$, TAYLOR WOOHOO.

$= y(t) + 2hy'(t) + 2h^2y''(t) + O(h^3) - y(t)/2 - hy'(t)/2 - h^2y''(t)/4 - O(h^3) - y(t)/2 - hy'(t) - 2h^2y''(t) - O(h^3) + hy'(t)/4 + h^2y''(t)/4 + O(h^3) - 3hy'(t)/4$ Simplify...

$= 2h^2y''(t) - h^2y''(t)/4 - 2h^2y''(t)  + h^2y''(t)/4 + O(h^3)$

$= O(h^3)$

Since we have eliminated all the $h, h^2$ terms and only $O(h^3)$ terms are left in the local truncation error, the method must be second order since it is zero stable, consistent. Therefore the global truncation error is one order lower and the method has global error $O(h^2)$.

In [6]:
#problem three
import numpy as np
from math import sqrt

def rk4(f, x0, y0, x1, n, t):
    vx = [0] * (n + 1)
    vy = [0] * (n + 1)
    h = (x1 - x0) / float(n)
    vx[0] = x = x0
    vy[0] = y = y0
    for i in range(1, n + 1):
        k1 = h * f(x, y)
        k2 = h * f(x + 0.5 * h, y + 0.5 * k1)
        k3 = h * f(x + 0.5 * h, y + 0.5 * k2)
        k4 = h * f(x + h, y + k3)
        vx[i] = x = x0 + i * h
        vy[i] = y = y + (k1 + k2 + k2 + k3 + k3 + k4) / 6
        if (vx[i] == x1):
            print("Endpoint", vx[i], "Estimate", vy[i], "Error", vy[i] - t(vx[i], vy[i]))
    return vx, vy

#part 1
def f(x, y):
    return (y/4)*(1-y/20)

def t(x, y):
    return 20/(1+19*np.exp(-x/4))
     
print("PROBLEM ONE")
for i in range(3, 11):
    print("Iteration with stepsize 1/", 2**i)
    vx, vy = rk4(f, 0, 1, 4, 4*2**i, t)
    
#It is clear that this is a fourth order method. Our final error is O(10^-15), and (1/1024)^4
#is O(10^-13), so we are in fact performing better than fourth order. Note that it begins to 
#run into issues near the end, indicating that numerical inaccuracy is beginning to decrease
#the order of the convergence. However until that point the error was decreasing at each step 
#by the ratio it should.

#part 2

#convert to two first order equations, let...
#u1 = y, u2 = y'

#initial conditions are u1(1) = 1/2, u2(1) = 1/4

#substitute variables, so now we have u1' = u2, y'' = (-2/x)(u1u2)

#modify the rk4 method accordingly

def rk42(f, g, x0, u10, u20, x1, n, t):
    vx = [0] * (n + 1)
    vu1 = [0] * (n + 1)
    vu2 = [0] * (n + 1)
    h = (x1 - x0) / float(n)
    vx[0] = x = x0
    vu1[0] = u1 = u10
    vu2[0] = u2 = u20
    for i in range(1, n + 1):
        k1 = h * f(x, u1, u2)
        j1 = h * g(u2)
        k2 = h * f(x + 0.5 * h, u1 + 0.5 * j1, u2 + 0.5 * k1)
        j2 = h * g(u2 + 0.5 * k1)
        k3 = h * f(x + 0.5 * h, u1 + 0.5 * j2, u2 + 0.5 * k2)
        j3 = h * g(u2 + 0.5 * k2)
        k4 = h * f(x + h, u1 + j3, u2 + k3)
        j4 = h * g(u2 + k3)
        vx[i] = x = x0 + i * h
        vu1[i] = u1 = u1 + (j1 + j2 + j2 + j3 + j3 + j4) / 6
        #print((k1 + k2 + k2 + k3 + k3 + k4) / 6)
        vu2[i] = u2 = u2 + (k1 + k2 + k2 + k3 + k3 + k4) / 6
        if (vx[i] == x1):
            print("Endpoint", vx[i], "Estimate", vu1[i], "Error", vu1[i] - t(vx[i], vu2[i]))
    return vx, vu1, vu2

def f(x, u1, u2):
    return (-2/x)*u1*u2

def g(u2):
    return u2

def t(x, y):
    return x/(1+x)
     
print("PROBLEM TWO")
for i in range(3, 11):
    print("Iteration with stepsize 1/", 2**i)
    vx, vu1, vu2 = rk42(f, g, 1, 1/2, 1/4, 2, 2*2**i, t)
#(j1 + j2 + j2 + j3 + j3 + j4)

#We can see here that this method too follows the correct pattern, running into the same 
#difficulties as the second problem once the errors in numerical precision begin to dominate.



PROBLEM ONE
Iteration with stepsize 1/ 8
Endpoint 4.0 Estimate 2.5032199488329265 Error -1.1133780120076153e-08
Iteration with stepsize 1/ 16
Endpoint 4.0 Estimate 2.503219959263102 Error -7.036047300346127e-10
Iteration with stepsize 1/ 32
Endpoint 4.0 Estimate 2.5032199599224874 Error -4.4219294892400285e-11
Iteration with stepsize 1/ 64
Endpoint 4.0 Estimate 2.503219959963935 Error -2.7715607586742408e-12
Iteration with stepsize 1/ 128
Endpoint 4.0 Estimate 2.5032199599665375 Error -1.6919798895287386e-13
Iteration with stepsize 1/ 256
Endpoint 4.0 Estimate 2.503219959966696 Error -1.0658141036401503e-14
Iteration with stepsize 1/ 512
Endpoint 4.0 Estimate 2.503219959966707 Error 4.440892098500626e-16
Iteration with stepsize 1/ 1024
Endpoint 4.0 Estimate 2.5032199599667093 Error 2.6645352591003757e-15
PROBLEM TWO
Iteration with stepsize 1/ 8
Endpoint 2.0 Estimate 0.6666666654791819 Error -1.1874847771764507e-09
Iteration with stepsize 1/ 16
Endpoint 2.0 Estimate 0.6666666665930545 E